# L1 Language Models, the Chat Format and Tokens

## Configuration
#### Chargement de la clé API et des librairies nécessaires

In [ ]:
import tiktoken
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0,
    )
    return response.choices[0].message.content

**Note**: Tous les notebooks suivants utilisent la version `0.27.0` de la bibliothèque d'OpenAI. 

Pour utiliser la version `1.0.0`, il faut utiliser la version suivante : 

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Interroger le modèle et obtenir la réponse

In [ ]:
response = get_completion("Quelle est la capitale de la France ?")

In [ ]:
print(response)

## Tokens

In [ ]:
response = get_completion("Prenez les lettres de 'lollipop' et inversez-les.")
print(response)

"lollipop" in reverse should be "popillol"

In [ ]:
response = get_completion("""Prenez les lettres de l-o-l-l-i-p-o-p et inversez-les.""", "gpt-3.5-turbo-1106")

In [ ]:
response

## Fonction helper (chat format)

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

In [ ]:
messages =  [  
{'role':'system', 
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants."""},    
{'role':'user', 
 'content':"""Ecrivez une histoire sur une carotte heureuse."""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
# length
messages =  [  
{'role':'system',
 'content':'Toutes vos réponses doivent être d"une seule phrase.'},    
{'role':'user',
 'content':'Ecrivez une histoire sur une carotte heureuse.'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

In [ ]:
# combined
messages =  [  
{'role':'system',
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants.
Toutes vos réponses doivent être d"une seule phrase."""},    
{'role':'user',
 'content':"""Ecrivez une histoire sur une carotte heureuse."""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In [ ]:
def get_completion_and_token_count(messages, 
                                   model="gpt-4o-mini", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message.content
    
    token_dict = {
'prompt_tokens':response.usage.prompt_tokens,
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [ ]:
messages = [
{'role':'system', 
 'content':"""Vous êtes un assistant qui réponds en prenant le style d'un conteur pour enfants."""},    
{'role':'user',
 'content':"""Ecrivez une très court poème sur une carotte heureuse."""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [ ]:
print(response)

In [ ]:
print(token_dict)

#### Notes sur l'utilisation de l'API OpenAI en dehors de cette salle de classe.

Pour installer la bibliothèque Python OpenAI :
```
!pip install openai
```

La bibliothèque doit être configurée avec la clé secrète de votre compte, qui est disponible sur le [site web](https://platform.openai.com/account/api-keys). 

Vous pouvez soit la définir comme variable d'environnement `OPENAI_API_KEY` avant d'utiliser la bibliothèque :
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Ou, définir `openai.api_key` :

```
import openai
openai.api_key = "sk-..."
```

#### Remarque complémentaire
- Dans ce cours nous utiliserons les antislashs `\` pour que le texte rentre dans l'écran sans utiliser une nouvelle ligne '\n'.
- GPT-4 n'est pas vraiment affecté par l'insertion de caractères de nouvelle ligne ou non. Mais en travaillant avec des LLM en général, vous pouvez considérer si les caractères de nouvelle ligne dans votre prompt peuvent affecter les performances du modèle.